# Illinois Scraping 

In [1]:
# set up 
import pandas as pd 
import requests 
import bs4
import numpy as np
import time 
import re
from tika import parser
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.select import Select
from io import BytesIO
from urllib.request import urlopen

from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all"

%xmode Minimal

Exception reporting mode: Minimal


In [1]:
# # set headers
# header_list = {'User-Agent': REDACTED,
#               'Accept-Language': 'en-US,en;q=0.9'}

In [3]:
# state - UPDATE EACH TIME
state = 'illinois'

In [ ]:
# review of robots.txt - no restrictions

In [ ]:
# test = pd.read_csv(f'{state}_2023.csv')
# test.head()

## Scrape Links 
`Selenium`

### Test on one link

 `Beautiful Soup` **Test**

In [4]:
# access webpage
link = 'https://gov.illinois.gov/newsroom/all-news.html'
server_response = requests.get(link, header_list)
server_response

<Response [200]>

In [7]:
# parse and isolate content - DIDN'T WORK 
soup = bs4.BeautifulSoup(server_response.content, features="html.parser")
results = soup.find_all('div', class_ = 'cmp-news-feed__text')
results

# title = [i.find('a').text for i in results]
# link = [i.find('a')['href'] for i in results]
# date = [i.find('span', class_ ='cmp-news-feed__type').text for i in results]

[]

### Scrape all links

In [20]:
# scrape all 
driver = webdriver.Chrome()
driver.get('https://gov.illinois.gov/newsroom/all-news.html')

wait = WebDriverWait(driver, 10)

wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'cmp-news-feed__text')))

results = driver.find_elements(By.CLASS_NAME, 'cmp-news-feed__text')

results_list = [{'title': i.find_element(By.TAG_NAME, 'a').text,
                'date': i.find_element(By.TAG_NAME, 'span').text, 
                'link': i.find_element(By.TAG_NAME, 'a').get_attribute('href')} for i in results]

df = pd.DataFrame.from_dict(results_list)

driver.close() 

[<selenium.webdriver.remote.webelement.WebElement (session="2cafe71a55db32e9af9289a069ba1d80", element="1C30C952D0C1AD88572C78E3281B4453_element_21")>,
 <selenium.webdriver.remote.webelement.WebElement (session="2cafe71a55db32e9af9289a069ba1d80", element="1C30C952D0C1AD88572C78E3281B4453_element_22")>,
 <selenium.webdriver.remote.webelement.WebElement (session="2cafe71a55db32e9af9289a069ba1d80", element="1C30C952D0C1AD88572C78E3281B4453_element_23")>,
 <selenium.webdriver.remote.webelement.WebElement (session="2cafe71a55db32e9af9289a069ba1d80", element="1C30C952D0C1AD88572C78E3281B4453_element_24")>,
 <selenium.webdriver.remote.webelement.WebElement (session="2cafe71a55db32e9af9289a069ba1d80", element="1C30C952D0C1AD88572C78E3281B4453_element_25")>,
 <selenium.webdriver.remote.webelement.WebElement (session="2cafe71a55db32e9af9289a069ba1d80", element="1C30C952D0C1AD88572C78E3281B4453_element_26")>,
 <selenium.webdriver.remote.webelement.WebElement (session="2cafe71a55db32e9af9289a069ba

In [21]:
df.head()
df.info()

,title,date,link
0,"Gov. Pritzker Advances Fiscal Responsibility, ...","Press Release - Wednesday, February 21, 2024",https://gov.illinois.gov/newsroom/press-releas...
1,Gov. Pritzker Announces One Appointment to Boa...,"Press Release - Friday, February 16, 2024",https://gov.illinois.gov/newsroom/press-releas...
2,Seasonal Respiratory Viruses Remain at Low Lev...,"Press Release - Friday, February 16, 2024",https://gov.illinois.gov/newsroom/press-releas...
3,State and County Announce $250 Million Commitm...,"Press Release - Thursday, February 15, 2024",https://gov.illinois.gov/newsroom/press-releas...
4,Governor Pritzker Awards Over $18 Million in G...,"Press Release - Thursday, February 15, 2024",https://gov.illinois.gov/newsroom/press-releas...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8588 entries, 0 to 8587
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   8588 non-null   object
 1   date    8588 non-null   object
 2   link    8588 non-null   object
dtypes: object(3)
memory usage: 201.4+ KB


In [23]:
df.tail()

,title,date,link
8583,Blagojevich takes action to protect consumers ...,"Press Release - Wednesday, August 20, 2003",https://gov.illinois.gov/newsroom/press-releas...
8584,Governor signs legislation to help ease state ...,"Press Release - Wednesday, August 20, 2003",https://gov.illinois.gov/newsroom/press-releas...
8585,Blagojevich signs law to protect the public fr...,"Press Release - Tuesday, August 19, 2003",https://gov.illinois.gov/newsroom/press-releas...
8586,GOVERNOR NAMES DEPUTY CHIEF OF STAFF FOR EDUCA...,"Press Release - Monday, August 18, 2003",https://gov.illinois.gov/newsroom/press-releas...
8587,Governor Signs Bill to Attract Film Industry B...,"Press Release - Monday, August 18, 2003",https://gov.illinois.gov/newsroom/press-releas...


In [ ]:
# subset to 2023 - this pulled 10 years 

In [48]:
# test date cleaning method 
text = 'Press Release - Wednesday, February 21, 2024'
def clean_date(text):
    result = text.split(',')
    date_clean = (result[1] + "," + result[2]).strip()
    return date_clean

clean_date(text)

'February 21, 2024'

In [49]:
# create clean date column 
df['date_cl']  = [clean_date(i) for i in df.date]

In [50]:
df.head()

,title,date,link,date_cl
0,"Gov. Pritzker Advances Fiscal Responsibility, ...","Press Release - Wednesday, February 21, 2024",https://gov.illinois.gov/newsroom/press-releas...,"February 21, 2024"
1,Gov. Pritzker Announces One Appointment to Boa...,"Press Release - Friday, February 16, 2024",https://gov.illinois.gov/newsroom/press-releas...,"February 16, 2024"
2,Seasonal Respiratory Viruses Remain at Low Lev...,"Press Release - Friday, February 16, 2024",https://gov.illinois.gov/newsroom/press-releas...,"February 16, 2024"
3,State and County Announce $250 Million Commitm...,"Press Release - Thursday, February 15, 2024",https://gov.illinois.gov/newsroom/press-releas...,"February 15, 2024"
4,Governor Pritzker Awards Over $18 Million in G...,"Press Release - Thursday, February 15, 2024",https://gov.illinois.gov/newsroom/press-releas...,"February 15, 2024"


In [52]:
# subset to 2023 only 
df['date_clean'] = pd.to_datetime(df['date_cl'])

df_23 = df[(df['date_clean'] >= '2023-01-01')
                     & (df['date_clean'] <= '2023-12-31')].copy()


In [56]:
df.to_csv(f'{state}_links.csv', index = False)

In [53]:
combined = df_23.drop(columns = ['date', 'date_cl']).copy()

In [55]:
combined.info()
combined.head()
combined.tail()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 533 entries, 55 to 587
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   title       533 non-null    object        
 1   link        533 non-null    object        
 2   date_clean  533 non-null    datetime64[ns]
dtypes: datetime64[ns](1), object(2)
memory usage: 16.7+ KB


,title,link,date_clean
55,Gov. Pritzker Celebrates 2023 Executive and Le...,https://gov.illinois.gov/newsroom/press-releas...,2023-12-29
56,Paid Leave for All Workers Act Takes Effect Ja...,https://gov.illinois.gov/newsroom/press-releas...,2023-12-27
57,State Deploys Additional Resources to Support ...,https://gov.illinois.gov/newsroom/press-releas...,2023-12-24
58,New Laws Taking Effect January 1st Expand Leav...,https://gov.illinois.gov/newsroom/press-releas...,2023-12-21
59,Governor Pritzker Announces Federal Recompete ...,https://gov.illinois.gov/newsroom/press-releas...,2023-12-20


,title,link,date_clean
583,Honoring Illinois' Fallen,https://gov.illinois.gov/newsroom/press-releas...,2023-01-06
584,Governor Pritzker Announces Second Term Agency...,https://gov.illinois.gov/newsroom/press-releas...,2023-01-06
585,Gov. Pritzker Visits Ball Elementary School to...,https://gov.illinois.gov/newsroom/press-releas...,2023-01-05
586,"State Receives Donation of Large, Modern Offic...",https://gov.illinois.gov/newsroom/press-releas...,2023-01-04
587,Gov. Pritzker Highlights Investments in Early ...,https://gov.illinois.gov/newsroom/press-releas...,2023-01-04


In [57]:
# safety save 
combined.to_csv(f'{state}_links_23.csv', index = False)

## Scrape content from links
`Beautiful Soup`

### Test with one link

In [77]:
# define link scraping 
def get_illinois_content(link, df, header_list):
    try: 
        # check if content has already been pulled
        result = df.loc[df.link == link, 'content'].notnull().all()
        if result:
            return f'content already existing for {link}' 
        else: 
            try: 
                df['content']
                server_response = requests.get(link, headers = header_list)
                soup_link = bs4.BeautifulSoup(server_response.content, features="html.parser")
                result = soup_link.find('div', 
                                        class_ ='press-release aem-GridColumn--default--none aem-GridColumn--phone--12 aem-GridColumn--default--9 aem-GridColumn--phone--newline aem-GridColumn aem-GridColumn--offset--phone--0 aem-GridColumn--offset--default--0').find_all('p')
                string_result = [str(i.text) for i in result]
                string_result_clean = [i.replace(u'\xa0', u'') for i in string_result]
                string_result_clean = [i.replace(u'\n', u'') for i in string_result_clean] 
                result_merged = " ".join(string_result_clean)
                
                df.loc[df.link == link, 'content'] = result_merged
                return f"success: content added from {link}"
            except: 
                pass
    except Exception as e: 
        return f"unable to gather content from {link} due to {e}"
    #return df_updated 

In [59]:
# access webpage
link = 'https://gov.illinois.gov/newsroom/press-release.29645.html'
server_response = requests.get(link, headers = header_list)
server_response

<Response [200]>

In [76]:
# parse and isolate content
soup_link = bs4.BeautifulSoup(server_response.content, features="html.parser")
result = soup_link.find('div', 
                        class_ ='press-release aem-GridColumn--default--none aem-GridColumn--phone--12 aem-GridColumn--default--9 aem-GridColumn--phone--newline aem-GridColumn aem-GridColumn--offset--phone--0 aem-GridColumn--offset--default--0').find_all('p')
string_result = [str(i.text) for i in result]
string_result_clean = [i.replace(u'\xa0', u'') for i in string_result]
string_result_clean = [i.replace(u'\n', u'') for i in string_result_clean] 
result_merged = " ".join(string_result_clean)
result_merged

['\nBuilding on a strong team of diverse experts in their fields, Governor JB Pritzker announced the following appointment in his administrationCapital Development BoardEileen Rhodes will continue to serve as a Member of the Capital Development Board.* Rhodes has over thirty years of residential property management and development experience. She currently serves as President for East Lake Management Group. Rhodes manages 10,000 housing units and oversees social services for senior buildings, single room occupancy buildings, and a mixed income rental development. Prior to joining East Lake Management, Rhodes was the Program Manager with Chicago Housing Authority and Director of Development with Chicago Dwellings Association. Rhodes is an active member in her community, where she is a board chair for Chicago Reader NFP and Illinois Capital Development Board; and is the owner of a Non-Profit Art Gallery Blanc Gallery. Eileen Rhodes received her Bachelor of Science from Georgetown Univers

'Building on a strong team of diverse experts in their fields, Governor JB Pritzker announced the following appointment in his administrationCapital Development BoardEileen Rhodes will continue to serve as a Member of the Capital Development Board.* Rhodes has over thirty years of residential property management and development experience. She currently serves as President for East Lake Management Group. Rhodes manages 10,000 housing units and oversees social services for senior buildings, single room occupancy buildings, and a mixed income rental development. Prior to joining East Lake Management, Rhodes was the Program Manager with Chicago Housing Authority and Director of Development with Chicago Dwellings Association. Rhodes is an active member in her community, where she is a board chair for Chicago Reader NFP and Illinois Capital Development Board; and is the owner of a Non-Profit Art Gallery Blanc Gallery. Eileen Rhodes received her Bachelor of Science from Georgetown University

### Scrape content from all links

In [79]:
# add blank content column
combined['content'] = np.nan 

In [80]:
# test on 5 to see if it works
for i in combined.link[0:6]:
     get_illinois_content(i, combined, header_list)
    
combined.head()

'success: content added from https://gov.illinois.gov/newsroom/press-release.29470.html'

'success: content added from https://gov.illinois.gov/newsroom/press-release.29457.html'

'success: content added from https://gov.illinois.gov/newsroom/press-release.29453.html'

'success: content added from https://gov.illinois.gov/newsroom/press-release.29444.html'

'success: content added from https://gov.illinois.gov/newsroom/press-release.29442.html'

,title,link,date_clean,content
55,Gov. Pritzker Celebrates 2023 Executive and Le...,https://gov.illinois.gov/newsroom/press-releas...,2023-12-29,Accomplishments include historic investments i...
56,Paid Leave for All Workers Act Takes Effect Ja...,https://gov.illinois.gov/newsroom/press-releas...,2023-12-27,Workers will begin earning up to 40 hours of p...
57,State Deploys Additional Resources to Support ...,https://gov.illinois.gov/newsroom/press-releas...,2023-12-24,"At request of City of Chicago, State temporari..."
58,New Laws Taking Effect January 1st Expand Leav...,https://gov.illinois.gov/newsroom/press-releas...,2023-12-21,Families impacted by violence afforded job sec...
59,Governor Pritzker Announces Federal Recompete ...,https://gov.illinois.gov/newsroom/press-releas...,2023-12-20,NaN


In [81]:
# do for all
for i in combined.link:
     get_illinois_content(i, combined, header_list)

'content already existing for https://gov.illinois.gov/newsroom/press-release.29470.html'

'content already existing for https://gov.illinois.gov/newsroom/press-release.29457.html'

'content already existing for https://gov.illinois.gov/newsroom/press-release.29453.html'

'content already existing for https://gov.illinois.gov/newsroom/press-release.29444.html'

'content already existing for https://gov.illinois.gov/newsroom/press-release.29442.html'

...

'success: content added from https://gov.illinois.gov/newsroom/press-release.25874.html'

'success: content added from https://gov.illinois.gov/newsroom/press-release.25875.html'

'success: content added from https://gov.illinois.gov/newsroom/press-release.25870.html'

'success: content added from https://gov.illinois.gov/newsroom/press-release.25864.html'

'success: content added from https://gov.illinois.gov/newsroom/press-release.25865.html'

## Validate and Clean

In [84]:
#check that content was pulled accurately
missing = combined[combined.content.isna()]

In [85]:
# retry missing content
for i in missing.link:
    get_illinois_content(i, combined, header_list)

'success: content added from https://gov.illinois.gov/newsroom/press-release.29441.html'

'success: content added from https://gov.illinois.gov/newsroom/press-release.29427.html'

'success: content added from https://gov.illinois.gov/newsroom/press-release.29428.html'

'success: content added from https://gov.illinois.gov/newsroom/press-release.29424.html'

'success: content added from https://gov.illinois.gov/newsroom/press-release.29419.html'

...

'success: content added from https://gov.illinois.gov/newsroom/press-release.27246.html'

'success: content added from https://gov.illinois.gov/newsroom/press-release.27248.html'

'success: content added from https://gov.illinois.gov/newsroom/press-release.26715.html'

'success: content added from https://gov.illinois.gov/newsroom/press-release.26562.html'

'success: content added from https://gov.illinois.gov/newsroom/press-release.26137.html'

In [91]:
#check that content was pulled accurately
combined[combined.content.isna()]

,title,link,date_clean,content
101,"IDPH Marks U.S. Antibiotic Awareness Week, Nov...",https://gov.illinois.gov/newsroom/press-releas...,2023-11-16,NaN


In [95]:
# issue with link 101
# get_illinois_content(combined.link[101], combined, header_list)
combined.link[101]
# manual review of link is it doesn't work so let's drop

combined_2023 = combined.drop([101])

combined.shape[0]
combined_2023.shape[0]

'https://gov.illinois.gov/newsroom/press-release.27306.html'

533

532

In [97]:
combined_2023.head()
combined_2023.tail()

,title,link,date_clean,content
55,Gov. Pritzker Celebrates 2023 Executive and Le...,https://gov.illinois.gov/newsroom/press-releas...,2023-12-29,Accomplishments include historic investments i...
56,Paid Leave for All Workers Act Takes Effect Ja...,https://gov.illinois.gov/newsroom/press-releas...,2023-12-27,Workers will begin earning up to 40 hours of p...
57,State Deploys Additional Resources to Support ...,https://gov.illinois.gov/newsroom/press-releas...,2023-12-24,"At request of City of Chicago, State temporari..."
58,New Laws Taking Effect January 1st Expand Leav...,https://gov.illinois.gov/newsroom/press-releas...,2023-12-21,Families impacted by violence afforded job sec...
59,Governor Pritzker Announces Federal Recompete ...,https://gov.illinois.gov/newsroom/press-releas...,2023-12-20,Two U.S. Department of Commerce's Economic Dev...


,title,link,date_clean,content
583,Honoring Illinois' Fallen,https://gov.illinois.gov/newsroom/press-releas...,2023-01-06,The Department of Central Management Services ...
584,Governor Pritzker Announces Second Term Agency...,https://gov.illinois.gov/newsroom/press-releas...,2023-01-06,Springfield—As Governor Pritzker concludes his...
585,Gov. Pritzker Visits Ball Elementary School to...,https://gov.illinois.gov/newsroom/press-releas...,2023-01-05,"SANGAMON COUNTY — Today, Governor JB Pritzker ..."
586,"State Receives Donation of Large, Modern Offic...",https://gov.illinois.gov/newsroom/press-releas...,2023-01-04,"MARION, IL - Governor JB Pritzker and the Illi..."
587,Gov. Pritzker Highlights Investments in Early ...,https://gov.illinois.gov/newsroom/press-releas...,2023-01-04,"CHICAGO — Today, Governor JB Pritzker visited ..."


In [98]:
print(f"pulled {str(len(combined_2023))} documents from {state} for 2023")

pulled 532 documents from illinois for 2023


## Export 

In [99]:
# export
combined_2023.to_csv(f'{state}_2023.csv', index = False)

In [100]:
test = pd.read_csv(f'{state}_2023.csv')
test.head()

,title,link,date_clean,content
0,Gov. Pritzker Celebrates 2023 Executive and Le...,https://gov.illinois.gov/newsroom/press-releas...,2023-12-29,Accomplishments include historic investments i...
1,Paid Leave for All Workers Act Takes Effect Ja...,https://gov.illinois.gov/newsroom/press-releas...,2023-12-27,Workers will begin earning up to 40 hours of p...
2,State Deploys Additional Resources to Support ...,https://gov.illinois.gov/newsroom/press-releas...,2023-12-24,"At request of City of Chicago, State temporari..."
3,New Laws Taking Effect January 1st Expand Leav...,https://gov.illinois.gov/newsroom/press-releas...,2023-12-21,Families impacted by violence afforded job sec...
4,Governor Pritzker Announces Federal Recompete ...,https://gov.illinois.gov/newsroom/press-releas...,2023-12-20,Two U.S. Department of Commerce's Economic Dev...
